In [2]:
import torch
import torchvision
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os

In [3]:
import random
class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None, percent=100):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = os.listdir(root_dir)
        self.class_to_idx = {cls: idx for idx, cls in enumerate(self.classes)}
        self.images = self._load_images(percent)

    def _load_images(self, percent):
        images = []
        for cls in self.classes:
            class_dir = os.path.join(self.root_dir, cls)
            class_images = [f for f in os.listdir(class_dir) if f.endswith('.jpg')]
            if percent < 100:
                num_images = int(len(class_images) * (percent / 100.0))
                class_images = random.sample(class_images, num_images)
            for img_name in class_images:
                img_path = os.path.join(class_dir, img_name)
                images.append((img_path, self.class_to_idx[cls]))
        return images

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path, label = self.images[idx]
        img = Image.open(img_path).convert('RGB')
        if self.transform:
            img = self.transform(img)
        return img, label

In [4]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

dataset = CustomDataset(root_dir='/kaggle/input/birds-and-squirrels/CV_data_v2/CV_data', transform=transform, percent=100)

In [5]:
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# Define data loaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [6]:
train_size, test_size

(6878, 1720)

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model = CustomCNN(num_classes=len(dataset.classes)).to(device)

model = torchvision.models.resnet18(pretrained=True)
model.fc = torch.nn.Linear(model.fc.in_features, len(dataset.classes))
model = model.to(device)


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 140MB/s] 


In [9]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [10]:
import time

# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    start_time = time.time()  # Record start time of epoch
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_accuracy = correct / total * 100
    end_time = time.time()  # Record end time of epoch
    epoch_time = end_time - start_time
    minutes = int(epoch_time // 60)
    seconds = int(epoch_time % 60)

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%, Time: {minutes} minutes {seconds} seconds")

Epoch [1/10], Loss: 0.1183, Accuracy: 95.75%, Time: 1 minutes 17 seconds
Epoch [2/10], Loss: 0.0522, Accuracy: 98.17%, Time: 0 minutes 29 seconds
Epoch [3/10], Loss: 0.0395, Accuracy: 98.59%, Time: 0 minutes 29 seconds
Epoch [4/10], Loss: 0.0226, Accuracy: 99.19%, Time: 0 minutes 28 seconds
Epoch [5/10], Loss: 0.0322, Accuracy: 98.81%, Time: 0 minutes 29 seconds
Epoch [6/10], Loss: 0.0271, Accuracy: 99.20%, Time: 0 minutes 28 seconds
Epoch [7/10], Loss: 0.0147, Accuracy: 99.51%, Time: 0 minutes 29 seconds
Epoch [8/10], Loss: 0.0363, Accuracy: 98.82%, Time: 0 minutes 28 seconds
Epoch [9/10], Loss: 0.0061, Accuracy: 99.87%, Time: 0 minutes 30 seconds
Epoch [10/10], Loss: 0.0016, Accuracy: 99.99%, Time: 0 minutes 29 seconds


In [11]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy on test set: {(correct / total) * 100:.2f}%")

Accuracy on test set: 99.71%


In [12]:
import torch
from torchvision import transforms
from PIL import Image

# Load the image and apply transformations

def predict_image(image_path, model):

    image = Image.open(image_path).convert('RGB')
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])
    input_image = transform(image).unsqueeze(0)  # Add batch dimension

    # Move input tensor to the device (GPU if available)
    input_image = input_image.to(device)

    # Perform inference
    model.eval()
    with torch.no_grad():
        output = model(input_image)

    # Get predicted class probabilities and class index
    probabilities = torch.softmax(output, dim=1)[0]
    predicted_class_index = torch.argmax(probabilities).item()

    # Map class index to class label
    class_labels = dataset.classes
    predicted_class_label = class_labels[predicted_class_index]

    return predicted_class_label, probabilities[predicted_class_index].item()
#     print("Class probabilities:")
#     for i, prob in enumerate(probabilities):
#         print(f"{class_labels[i]}: {prob:.4f}")


In [22]:
dataset.classes

['Bird', 'Squirrel']

In [15]:
import glob

for test_img in glob.glob("/kaggle/input/aipi540-test/test/*"):
    actual_class = test_img.split("/")[-1].split("_")[0]
    
    pred_class = predict_image(test_img, model)
    
    print(f"Actual Class: {actual_class} | Predicted Class: {pred_class}")

Actual Class: bird | Predicted Class: Bird
Actual Class: squirrel | Predicted Class: Squirrel
Actual Class: bird | Predicted Class: Bird
Actual Class: squirrel | Predicted Class: Squirrel
Actual Class: bird | Predicted Class: Bird
Actual Class: squirrel | Predicted Class: Squirrel
Actual Class: bird | Predicted Class: Bird
Actual Class: squirrel | Predicted Class: Squirrel
Actual Class: bird | Predicted Class: Bird
Actual Class: bird | Predicted Class: Bird
Actual Class: squirrel | Predicted Class: Squirrel
Actual Class: squirrel | Predicted Class: Bird
Actual Class: squirrel | Predicted Class: Squirrel
Actual Class: bird | Predicted Class: Bird
Actual Class: squirrel | Predicted Class: Squirrel
Actual Class: squirrel | Predicted Class: Squirrel
Actual Class: squirrel | Predicted Class: Bird
Actual Class: bird | Predicted Class: Bird
Actual Class: bird | Predicted Class: Bird
Actual Class: bird | Predicted Class: Squirrel


In [21]:
# Define the file path to save the model
model_path = 'resnet18_custom_model.pth'

# Save the model
torch.save(model.state_dict(), model_path)

print(f"Model saved to {model_path}")

Model saved to resnet18_custom_model.pth


In [16]:
import pickle

model_path = 'model.pkl'
with open(model_path, 'wb') as f:
    pickle.dump(model, f)

In [22]:
model_loaded = torchvision.models.resnet18(pretrained=False)  # Initialize ResNet18 without pretraining
model_loaded.fc = torch.nn.Linear(model_loaded.fc.in_features, len(dataset.classes))  # Modify the fully connected layer
model_loaded = model_loaded.to(device)  # Move the model to the appropriate device (GPU or CPU)

# Load the saved state dictionary into the model
model_path = '/kaggle/working/resnet18_custom_model.pth'
model_loaded.load_state_dict(torch.load(model_path))

# Set the model to evaluation mode
model_loaded.eval()

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [17]:
with open(model_path, 'rb') as f:
    loaded_model = pickle.load(f)

In [18]:
import glob

for test_img in glob.glob("/kaggle/input/aipi540-test/test/*"):
    actual_class = test_img.split("/")[-1].split("_")[0]
    
    pred_class = predict_image(test_img, loaded_model)
    
    print(f"Actual Class: {actual_class} | Predicted Class: {pred_class}")

Actual Class: bird | Predicted Class: Bird
Actual Class: squirrel | Predicted Class: Squirrel
Actual Class: bird | Predicted Class: Bird
Actual Class: squirrel | Predicted Class: Squirrel
Actual Class: bird | Predicted Class: Bird
Actual Class: squirrel | Predicted Class: Squirrel
Actual Class: bird | Predicted Class: Bird
Actual Class: squirrel | Predicted Class: Squirrel
Actual Class: bird | Predicted Class: Bird
Actual Class: bird | Predicted Class: Bird
Actual Class: squirrel | Predicted Class: Squirrel
Actual Class: squirrel | Predicted Class: Bird
Actual Class: squirrel | Predicted Class: Squirrel
Actual Class: bird | Predicted Class: Bird
Actual Class: squirrel | Predicted Class: Squirrel
Actual Class: squirrel | Predicted Class: Squirrel
Actual Class: squirrel | Predicted Class: Bird
Actual Class: bird | Predicted Class: Bird
Actual Class: bird | Predicted Class: Bird
Actual Class: bird | Predicted Class: Squirrel


In [27]:
import glob

for test_img in glob.glob("/kaggle/input/test-2/*"):
#     actual_class = test_img.split("/")[-1].split("_")[0]
    
    pred_class = predict_image(test_img, model_loaded)
    
    print(f"Actual Class: {test_img} | Predicted Class: {pred_class}")

Actual Class: /kaggle/input/test-2/small_bird_2_day.jpg | Predicted Class: Bird
Actual Class: /kaggle/input/test-2/squirrel_2.jpg | Predicted Class: Squirrel
Actual Class: /kaggle/input/test-2/night_bird_1.jpg | Predicted Class: Bird
Actual Class: /kaggle/input/test-2/parrot_3_day.jpg | Predicted Class: Bird
Actual Class: /kaggle/input/test-2/night_bird_3.jpg | Predicted Class: Bird
Actual Class: /kaggle/input/test-2/squirrel_4_day.jpg | Predicted Class: Squirrel
Actual Class: /kaggle/input/test-2/bird_group_1_day.jpg | Predicted Class: Squirrel
Actual Class: /kaggle/input/test-2/night_bird_2.jpg | Predicted Class: Bird
Actual Class: /kaggle/input/test-2/squirrel_1_dayjpg.jpg | Predicted Class: Squirrel
Actual Class: /kaggle/input/test-2/squirrel_1.jpg | Predicted Class: Squirrel
Actual Class: /kaggle/input/test-2/squirrel_5.jpg | Predicted Class: Squirrel
Actual Class: /kaggle/input/test-2/parrot_2.jpg | Predicted Class: Squirrel
Actual Class: /kaggle/input/test-2/parrot_2_day.jpg | P

In [21]:
torchvision.__version__

'0.16.2'